<a href="https://colab.research.google.com/github/timbosssds/ML/blob/main/VectorDB%20%26%20retrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Want to set up a vector db, so I can try 'MultiQueryRetriever'

In [ ]:
# source: https://python.langchain.com/docs/use_cases/question_answering/

In [22]:
# # Install required libraries
#!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers
#!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers

# -- Only below run
#!pip install langchain
#!pip install pypdf
#!pip install PyPDF2
#!pip install chromadb
#!pip install --upgrade chromadb

In [3]:
# -- Not run
#import bs4
#from langchain import hub
#from langchain.chat_models import ChatOpenAI
#from langchain.document_loaders import WebBaseLoader
#from langchain.embeddings import OpenAIEmbeddings
#from langchain.schema import StrOutputParser
#from langchain.schema.runnable import RunnablePassthrough
#from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.vectorstores import Chroma

In [31]:
# MileStone 1: Ingest, chunk, embed, store in df.
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# Not using atm
# # -- 3.0 Process text
# def convert_list_to_string_with_speech_marks(list_items):
#   """Converts a list to a string with speech marks at each end."""
#   string = " ".join(list_items)
#   string = f"\"{string}\""
#   return string
# list_items = texts
# text = convert_list_to_string_with_speech_marks(list_items)


# ------------ new (embed & store)
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Define the chunked text
chunked_text = texts

# Load the Sentence Transformer model
model = SentenceTransformer("paraphrase-mpnet-base-v2")

# Embed the chunked text
embeddings = model.encode(chunked_text)

# Convert embedding vectors into one-dimensional arrays
one_dimensional_embeddings = []
for embedding in embeddings:
    one_dimensional_embedding = np.ravel(embedding)
    one_dimensional_embeddings.append(one_dimensional_embedding)

# Combine the original text and embeddings into a DataFrame
data = {
    "original_text": chunked_text,
    "embeddings": one_dimensional_embeddings
}

df = pd.DataFrame(data)
display(df)

,original_text,embeddings
0,Financial \nServices \nGuide.\nDated 1 Octobe...,"[-0.0912222, -0.003018869, 0.038152117, -0.033..."
1,Services \nGuide.Our products and services\nO...,"[-0.033502605, -0.13524558, 0.0170168, -0.0555..."
2,When providing financial services that relate ...,"[-0.057890907, -0.12968607, 0.057180542, 0.018..."
3,Investments HoldCo Pty Limited ABN 64 644 660 ...,"[-0.0370728, -0.03843237, 0.02864407, 0.032806..."
4,Our staff are remunerated by salary that inclu...,"[-0.10325377, -0.19674908, -0.01208837, -0.006..."
5,"and cultural events, corporate promotional mer...","[0.0023660515, 0.072373606, 0.003049189, 0.025..."
6,"Insurance Partners Limited from us, the commis...","[-0.05016672, -0.054218188, 0.0806684, -0.0192..."
7,"Australian Prudential Regulation Authority, we...","[-0.041048624, 0.14313406, -0.072433785, 0.100..."
8,important we hear about it so we can make thin...,"[-0.08806811, -0.04257054, -0.017361928, -0.03..."
9,You will be advised of the details (including ...,"[-0.15787879, 0.014245284, 0.00045931916, 0.07..."


In [32]:
model = "deepset/roberta-base-squad2"

In [42]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load Sentence Transformer model
model = SentenceTransformer("paraphrase-mpnet-base-v2")

# Define chunked text
chunked_text = [
    "This is the first chunk of text.",
    "This is the second chunk of text.",
    "This is the third chunk of text."
]

# Embed the chunked text
embeddings = model.encode(chunked_text)

# Convert embedding vectors into one-dimensional arrays
one_dimensional_embeddings = []
for embedding in embeddings:
    one_dimensional_embedding = np.ravel(embedding)
    one_dimensional_embeddings.append(one_dimensional_embedding)

# Convert one-dimensional embeddings into a NumPy array
one_dimensional_embeddings_array = np.array(one_dimensional_embeddings)

# Create a Faiss index using the shape of the NumPy array
index = faiss.IndexFlatL2(one_dimensional_embeddings_array.shape[1])

# Add the embedding vectors to the Faiss index
index.add(one_dimensional_embeddings_array)

# Convert the Faiss index to a vector store
vectorstore = Chroma.from_index(index=index)

# Access the retriever object
retriever = vectorstore.as_retriever()


AttributeError: ignored

In [31]:
# Draft code

In [12]:
# # (not actually required now)
# # -- POC for above (embed)
# from sentence_transformers import SentenceTransformer
# # Define the text to embed
# text = texts
# # Load the Sentence Transformer model
# model = SentenceTransformer("paraphrase-mpnet-base-v2")
# # Embed the text into a vector
# embedding = model.encode(text)
# # Print the embedding vector
# print(embedding)


# # -- POC for above (embed + store)
# import pandas as pd
# import numpy as np
# from sentence_transformers import SentenceTransformer

# # Define the chunked text
# chunked_text = [
#     "This is the first chunk of text.",
#     "This is the second chunk of text.",
#     "This is the third chunk of text.",
# ]

# # Load the Sentence Transformer model
# model = SentenceTransformer("paraphrase-mpnet-base-v2")

# # Embed the chunked text
# embeddings = model.encode(chunked_text)

# # Convert embedding vectors into one-dimensional arrays
# one_dimensional_embeddings = []
# for embedding in embeddings:
#     one_dimensional_embedding = np.ravel(embedding)
#     one_dimensional_embeddings.append(one_dimensional_embedding)

# # Combine the original text and embeddings into a DataFrame
# data = {
#     "original_text": chunked_text,
#     "embeddings": one_dimensional_embeddings
# }

# df = pd.DataFrame(data)
# display(df)

[[-0.0912222  -0.00301887  0.03815212 ...  0.07174291 -0.06259535
   0.1135347 ]
 [-0.0335026  -0.13524558  0.0170168  ...  0.05752854 -0.05583623
   0.06697398]
 [-0.05789091 -0.12968607  0.05718054 ...  0.13090248  0.05814857
  -0.00228455]
 ...
 [-0.08806811 -0.04257054 -0.01736193 ...  0.07691211 -0.14527045
   0.02107514]
 [-0.15787879  0.01424528  0.00045932 ...  0.1147474  -0.09463678
   0.12607846]
 [-0.16931729  0.1199248   0.00443516 ...  0.0104755  -0.153561
   0.1347719 ]]


,original_text,embeddings
0,This is the first chunk of text.,"[0.09126054, -0.31641886, 0.020784846, -0.0246..."
1,This is the second chunk of text.,"[0.16886291, -0.22961397, -0.032136045, -0.028..."
2,This is the third chunk of text.,"[0.20351318, -0.37834385, 0.039949313, -0.0103..."
